In [9]:
import pandas as pd 

In [18]:
file=pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\notifications_india.csv")
file.head()

,notification_id,recipient_type,recipient_id,channel,subject,message,status,external_id,attempts,sent_at,created_at
0,N000001,driver,d0083,push,Reminder,Your route has been assigned. Check app.,queued,NaN,1,2025-11-13T10:20:24.758526,2025-11-13T11:09:24.758538
1,N000002,driver,d0024,email,Route Assignment,Your route has been assigned. Check app.,queued,NaN,2,2025-11-13T09:59:24.758865,2025-11-13T11:09:24.758875
2,N000003,driver,d0156,push,Route Assignment,Your route has been assigned. Check app.,queued,NaN,1,2025-11-13T10:18:24.759140,2025-11-13T11:09:24.759149
3,N000004,driver,d0001,sms,Route Assignment,Your route has been assigned. Check app.,sent,NaN,0,2025-11-13T09:44:24.759472,2025-11-13T11:09:24.759484
4,N000005,driver,d0107,sms,Reminder,Your route has been assigned. Check app.,delivered,NaN,0,2025-11-13T09:11:24.759742,2025-11-13T11:09:24.759749


In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from datetime import datetime

# ---------- FILE PATH ----------
BASE = r"C:\Users\ASUS\My_Learning\sat hack"

# ---------- LOAD CSV DATA ----------
trips = pd.read_csv(f"{BASE}\\trips_india.csv", parse_dates=['start_time','end_time','created_at'])
assignments = pd.read_csv(f"{BASE}\\assignments_india.csv", parse_dates=['scheduled_start','scheduled_end','created_at','updated_at'])
vehicles = pd.read_csv(f"{BASE}\\vehicle_agent_india_full.csv")
drivers = pd.read_csv(f"{BASE}\\driver_agent_india_full.csv")
environment = pd.read_csv(f"{BASE}\\environment_agent_india_full.csv", parse_dates=['time_window_start','time_window_end','last_updated'])
co2_aggregates = pd.read_csv(f"{BASE}\\co2_aggregates_india.csv")
fuel_predictions = pd.read_csv(f"{BASE}\\fuel_predictions_india.csv", parse_dates=['created_at'])
notifications = pd.read_csv(f"{BASE}\\notifications_india.csv")  # fuel alerts log

# ---------- MERGE DATA ----------
merge_keys = ['assignment_id', 'driver_id', 'vehicle_id', 'route_id']
df = trips.merge(assignments, on=merge_keys, how='left') \
          .merge(vehicles, on='vehicle_id', how='left') \
          .merge(drivers, on='driver_id', how='left') \
          .merge(environment, on='route_id', how='left') \
          .merge(co2_aggregates, left_on='trip_id', right_on='scope_id', how='left')

print("Merged DataFrame:")
display(df.head())

# ---------- FUEL EFFICIENCY ----------
df['fuel_efficiency_kmpl'] = df['distance_km'] / df['fuel_used_l']
df['efficiency_variance'] = df['fuel_efficiency_kmpl'] - df['baseline_kmpl']

# ---------- FUEL PREDICTION ----------
features = ['distance_km', 'duration_min']
if 'efficiency_multiplier' in df.columns:
    features.append('efficiency_multiplier')
if 'load_factor' in df.columns:
    features.append('load_factor')

X = df[features]
y = df['fuel_used_l']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
df['predicted_fuel_l'] = model.predict(X)

# ---------- CO2 CALCULATION ----------
df['predicted_co2_kg'] = df['predicted_fuel_l'] * df['emission_factor']

# ---------- REWARD CALCULATION ----------
df['reward_points'] = df['efficiency_variance'].apply(lambda x: max(0, x*10))

# ---------- DYNAMIC ALERTS ----------
vehicle_thresholds = {'van': 1.05, 'cargo-auto': 1.1, 'pickup':1.08, 'mini-truck':1.12, 'truck':1.15}
df['alert_threshold'] = df['vehicle_type'].map(vehicle_thresholds).fillna(1.1)

alerts = df[df['predicted_fuel_l'] > df['fuel_used_l'] * df['alert_threshold']]

if not alerts.empty:
    print("\nFuel Alerts:")
    display(alerts[['vehicle_id','driver_id','route_id','predicted_fuel_l','fuel_used_l','alert_threshold']])
    # Log alerts to notifications
    for _, row in alerts.iterrows():
        notifications = pd.concat([notifications, pd.DataFrame({
            'notification_id':[f"N{datetime.now().strftime('%f')}"],
            'recipient_type':['driver'],
            'recipient_id':[row['driver_id']],
            'channel':['push'],
            'subject':['Fuel Alert'],
            'message':[f"Trip {row['trip_id']} has higher than expected fuel consumption."],
            'status':['queued'],
            'external_id':[None],
            'attempts':[0],
            'sent_at':[None],
            'created_at':[datetime.now()]
        })], ignore_index=True)
    notifications.to_csv(f"{BASE}\\notifications_india.csv", index=False)
    print("Alerts logged to notifications_india.csv")

# ---------- FUEL AGENT REPORT ----------
fuel_report = df.groupby(['vehicle_id','driver_id']).agg({
    'fuel_used_l':'sum',
    'predicted_fuel_l':'sum',
    'predicted_co2_kg':'sum',
    'reward_points':'sum'
}).reset_index()

fuel_report.to_csv(f"{BASE}\\fuel_agent_report.csv", index=False)
print("\nFuel Agent report generated: fuel_agent_report.csv")
display(fuel_report.head())


Merged DataFrame:


,trip_id,assignment_id,driver_id,vehicle_id,route_id,start_time,end_time,distance_km,duration_min,fuel_before_l,...,time_modifier,last_updated,id,scope_type,scope_id,period_start,period_end,total_co2_kg,trips_count,created_at
0,T000001,A000417,d0199,v0038,R000359,2025-11-18 16:00:00,2025-11-18 17:56:00,35.22,116,39.29,...,1.57,2025-11-13 11:04:37.315377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T000002,A000399,d0129,v0128,R000295,2025-11-18 11:30:00,2025-11-18 12:41:00,25.75,71,46.34,...,1.48,2025-11-13 11:04:37.292664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T000003,A000241,d0120,v0051,R000643,2025-11-16 20:00:00,2025-11-16 21:00:00,15.71,60,21.32,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T000004,A000303,d0012,v0045,R000747,2025-11-17 11:30:00,2025-11-17 12:05:00,22.66,35,24.28,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T000005,A000278,d0088,v0015,R001102,2025-11-17 05:15:00,2025-11-17 05:21:00,2.76,6,47.43,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Fuel Alerts:


,vehicle_id,driver_id,route_id,predicted_fuel_l,fuel_used_l,alert_threshold
1,v0128,d0129,R000295,3.36468,2.992,1.12
2,v0051,d0120,R000643,1.57842,1.298,1.08
5,v0121,d0061,R000805,3.88276,3.353,1.10
12,v0095,d0141,R001028,2.97694,2.280,1.08
15,v0107,d0156,R000739,3.28550,2.865,1.08
...,...,...,...,...,...,...
454,v0113,d0163,R000577,5.11088,4.254,1.05
460,v0132,d0110,R000797,0.77121,0.405,1.05
461,v0132,d0110,R000797,0.77121,0.405,1.05
462,v0061,d0026,R000417,3.93003,2.409,1.12


Alerts logged to notifications_india.csv

Fuel Agent report generated: fuel_agent_report.csv


,vehicle_id,driver_id,fuel_used_l,predicted_fuel_l,predicted_co2_kg,reward_points
0,v0001,d0008,13.030,13.63092,25.898748,0
1,v0001,d0046,6.026,8.47850,16.109150,0
2,v0001,d0104,8.816,10.04586,19.087134,0
3,v0001,d0108,4.909,5.13978,9.765582,0
4,v0003,d0124,0.768,1.13897,2.631021,0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from datetime import datetime

# ---------- FILE PATH ----------
BASE = r"C:\Users\ASUS\My_Learning\sat hack"

# ---------- LOAD CSV DATA ----------
trips = pd.read_csv(f"{BASE}\\trips_india.csv", parse_dates=['start_time','end_time','created_at'])
assignments = pd.read_csv(f"{BASE}\\assignments_india.csv", parse_dates=['scheduled_start','scheduled_end','created_at','updated_at'])
vehicles = pd.read_csv(f"{BASE}\\vehicle_agent_india_full.csv")
drivers = pd.read_csv(f"{BASE}\\driver_agent_india_full.csv")
environment = pd.read_csv(f"{BASE}\\environment_agent_india_full.csv", parse_dates=['time_window_start','time_window_end','last_updated'])
co2_aggregates = pd.read_csv(f"{BASE}\\co2_aggregates_india.csv")
fuel_predictions = pd.read_csv(f"{BASE}\\fuel_predictions_india.csv", parse_dates=['created_at'])
notifications = pd.read_csv(f"{BASE}\\notifications_india.csv")

# ---------- MERGE DATA ----------
merge_keys = ['assignment_id', 'driver_id', 'vehicle_id', 'route_id']
df = trips.merge(assignments, on=merge_keys, how='left') \
          .merge(vehicles, on='vehicle_id', how='left') \
          .merge(drivers, on='driver_id', how='left') \
          .merge(environment, on='route_id', how='left') \
          .merge(co2_aggregates, left_on='trip_id', right_on='scope_id', how='left')

# ---------- FEATURE ENGINEERING ----------
df['fuel_efficiency_kmpl'] = df['distance_km'] / df['fuel_used_l']
df['efficiency_variance'] = df['fuel_efficiency_kmpl'] - df['baseline_kmpl']

# Driver & vehicle behavior features
behavior_features = ['avg_speed_kmph','speed_variation','acceleration_profile',
                     'braking_profile','idle_time_pct','efficiency_multiplier','load_factor']

# Environment features
env_features = ['slope_pct_avg','temp_celsius','humidity_pct','air_quality_index','fuel_modifier','time_modifier']

# Base trip features
trip_features = ['distance_km','duration_min']

# Final feature set
features = trip_features + behavior_features + env_features

# Fill missing values
for f in features:
    if f in df.columns:
        df[f] = df[f].fillna(df[f].median())
    else:
        df[f] = 0  # if missing column, fill zero

# ---------- FUEL PREDICTION ----------
X = df[features]
y = df['fuel_used_l']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
df['predicted_fuel_l'] = model.predict(X)

# ---------- CO2 CALCULATION ----------
df['predicted_co2_kg'] = df['predicted_fuel_l'] * df['emission_factor']

# ---------- REWARD CALCULATION ----------
df['reward_points'] = df['efficiency_variance'].apply(lambda x: max(0, x*10))

# ---------- DYNAMIC ALERTS ----------
vehicle_thresholds = {'van': 1.05, 'cargo-auto': 1.1, 'pickup':1.08, 'mini-truck':1.12, 'truck':1.15}
df['alert_threshold'] = df['vehicle_type'].map(vehicle_thresholds).fillna(1.1)

alerts = df[df['predicted_fuel_l'] > df['fuel_used_l'] * df['alert_threshold']]

if not alerts.empty:
    print("\nFuel Alerts:")
    display(alerts[['vehicle_id','driver_id','route_id','predicted_fuel_l','fuel_used_l','alert_threshold']])
    # Log alerts to notifications
    for _, row in alerts.iterrows():
        notifications = pd.concat([notifications, pd.DataFrame({
            'notification_id':[f"N{datetime.now().strftime('%f')}"],
            'recipient_type':['driver'],
            'recipient_id':[row['driver_id']],
            'channel':['push'],
            'subject':['Fuel Alert'],
            'message':[f"Trip {row['trip_id']} has higher than expected fuel consumption."],
            'status':['queued'],
            'external_id':[None],
            'attempts':[0],
            'sent_at':[None],
            'created_at':[datetime.now()]
        })], ignore_index=True)
    notifications.to_csv(f"{BASE}\\notifications_india.csv", index=False)
    print("Alerts logged to notifications_india.csv")

# ---------- FUEL AGENT REPORT ----------
fuel_report = df.groupby(['vehicle_id','driver_id']).agg({
    'fuel_used_l':'sum',
    'predicted_fuel_l':'sum',
    'predicted_co2_kg':'sum',
    'reward_points':'sum'
}).reset_index()

fuel_report.to_csv(f"{BASE}\\fuel_agent_report.csv", index=False)
print("\nFuel Agent report generated: fuel_agent_report.csv")
display(fuel_report.head())



Fuel Alerts:


,vehicle_id,driver_id,route_id,predicted_fuel_l,fuel_used_l,alert_threshold
1,v0128,d0129,R000295,3.506630,2.992,1.12
2,v0051,d0120,R000643,1.744995,1.298,1.08
4,v0015,d0088,R001102,0.531830,0.481,1.05
5,v0121,d0061,R000805,3.882330,3.353,1.10
12,v0095,d0141,R001028,3.253575,2.280,1.08
...,...,...,...,...,...,...
454,v0113,d0163,R000577,5.374775,4.254,1.05
460,v0132,d0110,R000797,0.997040,0.405,1.05
461,v0132,d0110,R000797,0.819185,0.405,1.05
462,v0061,d0026,R000417,3.944950,2.409,1.12


Alerts logged to notifications_india.csv

Fuel Agent report generated: fuel_agent_report.csv


,vehicle_id,driver_id,fuel_used_l,predicted_fuel_l,predicted_co2_kg,reward_points
0,v0001,d0008,13.030,12.225660,23.228754,0
1,v0001,d0046,6.026,9.304370,17.678303,0
2,v0001,d0104,8.816,9.892340,18.795446,0
3,v0001,d0108,4.909,5.455800,10.366020,0
4,v0003,d0124,0.768,1.343805,3.104190,0


In [26]:
import pandas as pd
import numpy as np

class FuelAgent:
    def __init__(self, trips, vehicles, fuel_preds=None):
        """
        trips: DataFrame containing trips (trip_id, vehicle_id, distance_km, etc.)
        vehicles: DataFrame containing vehicle info (vehicle_id, fuel_type, baseline_kmpl)
        fuel_preds: optional DataFrame with predicted fuel usage (assignment_id, predicted_fuel_l)
        """
        self.trips = trips.copy()
        self.vehicles = vehicles.copy()
        self.fuel_preds = fuel_preds.copy() if fuel_preds is not None else None

        # Live segments storage for mid-trip updates
        self.live_segments = pd.DataFrame(columns=['trip_id','fuel_used_l'])

        # Strip columns
        for df in [self.trips, self.vehicles, self.fuel_preds]:
            if df is not None:
                df.columns = df.columns.str.strip().str.lower()

        # Merge trips with vehicle info
        self._preprocess_data()

        # Compute initial fuel usage
        self._compute_fuel()

    # ---------------- Internal Preprocessing ----------------
    def _preprocess_data(self):
        self.vehicles['baseline_kmpl'] = self.vehicles['baseline_kmpl'].fillna(10)
        self.vehicles['fuel_efficiency_l_per_km'] = 1 / self.vehicles['baseline_kmpl']

        self.trips['distance_km'] = self.trips.get('distance_km', 0)
        self.trips = self.trips.merge(
            self.vehicles[['vehicle_id','fuel_efficiency_l_per_km']],
            on='vehicle_id', how='left'
        )
        self.trips['fuel_efficiency_l_per_km'] = self.trips['fuel_efficiency_l_per_km'].fillna(0.1)

        # Merge predicted fuel if provided
        if self.fuel_preds is not None:
            self.trips = self.trips.merge(
                self.fuel_preds[['trip_id','predicted_fuel_l']],
                on='trip_id', how='left'
            )
        else:
            self.trips['predicted_fuel_l'] = np.nan

    # ---------------- Compute fuel usage ----------------
    def _compute_fuel(self):
        df = self.trips.copy()

        # Add live segment updates if any
        if not self.live_segments.empty:
            live_agg = self.live_segments.groupby('trip_id')['fuel_used_l'].sum().reset_index()
            live_agg.rename(columns={'fuel_used_l':'fuel_used_l_live'}, inplace=True)
            df = df.merge(live_agg, on='trip_id', how='left')
            df['fuel_used_l'] = df.get('fuel_used_l_live').combine_first(df['distance_km']*df['fuel_efficiency_l_per_km'])
            df.drop(columns=['fuel_used_l_live'], inplace=True, errors='ignore')
        else:
            # Default calculation based on distance and efficiency
            df['fuel_used_l'] = df['distance_km'] * df['fuel_efficiency_l_per_km']

        self.fuel_data = df

    # ---------------- Public method to get fuel data ----------------
    def calculate_fuel(self):
        self._compute_fuel()
        return self.fuel_data

    # ---------------- Update live segment ----------------
    def update_segment(self, trip_id, fuel_used_l):
        """
        Add a mid-trip fuel update (live segment)
        """
        segment = {'trip_id': trip_id, 'fuel_used_l': fuel_used_l}
        self.live_segments = pd.concat([self.live_segments, pd.DataFrame([segment])], ignore_index=True)
        print(f"Live segment updated for trip {trip_id}")
        return self.calculate_fuel()


# ==============================
# DEMO
# ==============================
if __name__ == "__main__":
    trips = pd.DataFrame([
        {'trip_id':'T1','vehicle_id':'V1','distance_km':100},
        {'trip_id':'T2','vehicle_id':'V2','distance_km':150}
    ])
    vehicles = pd.DataFrame([
        {'vehicle_id':'V1','baseline_kmpl':12},
        {'vehicle_id':'V2','baseline_kmpl':10}
    ])
    fuel_preds = pd.DataFrame([
        {'trip_id':'T1','predicted_fuel_l':9},
        {'trip_id':'T2','predicted_fuel_l':16}
    ])

    agent = FuelAgent(trips, vehicles, fuel_preds)

    print("Initial Fuel Calculation:")
    print(agent.calculate_fuel()[['trip_id','distance_km','fuel_used_l','predicted_fuel_l']])

    # Update a live segment for T2
    agent.update_segment('T2', fuel_used_l=14)

    print("\nFuel Calculation After Live Update:")
    print(agent.calculate_fuel()[['trip_id','distance_km','fuel_used_l','predicted_fuel_l']])


Initial Fuel Calculation:
  trip_id  distance_km  fuel_used_l  predicted_fuel_l
0      T1          100     8.333333                 9
1      T2          150    15.000000                16
Live segment updated for trip T2

Fuel Calculation After Live Update:
  trip_id  distance_km fuel_used_l  predicted_fuel_l
0      T1          100    8.333333                 9
1      T2          150          14                16


In [27]:
import pandas as pd
import numpy as np
from datetime import datetime

class FuelAgent:
    def __init__(self, trips, vehicles, fuel_preds=None, vehicle_thresholds=None):
        """
        trips: DataFrame with trip info (trip_id, vehicle_id, distance_km)
        vehicles: DataFrame with vehicle info (vehicle_id, baseline_kmpl)
        fuel_preds: optional DataFrame with predicted fuel usage
        vehicle_thresholds: dict mapping vehicle_type -> alert multiplier
        """
        self.trips = trips.copy()
        self.vehicles = vehicles.copy()
        self.fuel_preds = fuel_preds.copy() if fuel_preds is not None else None
        self.vehicle_thresholds = vehicle_thresholds or {'default': 1.1}

        self.notifications = pd.DataFrame(columns=[
            'notification_id','recipient_type','recipient_id','channel',
            'subject','message','status','created_at'
        ])

        self.live_segments = pd.DataFrame(columns=['trip_id','fuel_used_l'])

        # Standardize columns
        for df in [self.trips, self.vehicles, self.fuel_preds]:
            if df is not None:
                df.columns = df.columns.str.strip().str.lower()

        self._preprocess_data()
        self._compute_fuel()

    # ---------------- Preprocessing ----------------
    def _preprocess_data(self):
        self.vehicles['baseline_kmpl'] = self.vehicles['baseline_kmpl'].fillna(10)
        self.vehicles['fuel_efficiency_l_per_km'] = 1 / self.vehicles['baseline_kmpl']

        self.trips['distance_km'] = self.trips.get('distance_km', 0)
        self.trips = self.trips.merge(
            self.vehicles[['vehicle_id','fuel_efficiency_l_per_km','vehicle_type']],
            on='vehicle_id', how='left'
        )
        self.trips['fuel_efficiency_l_per_km'] = self.trips['fuel_efficiency_l_per_km'].fillna(0.1)
        self.trips['vehicle_type'] = self.trips['vehicle_type'].fillna('default')

        if self.fuel_preds is not None:
            self.trips = self.trips.merge(
                self.fuel_preds[['trip_id','predicted_fuel_l']],
                on='trip_id', how='left'
            )
        else:
            self.trips['predicted_fuel_l'] = np.nan

    # ---------------- Compute fuel ----------------
    def _compute_fuel(self):
        df = self.trips.copy()

        if not self.live_segments.empty:
            live_agg = self.live_segments.groupby('trip_id')['fuel_used_l'].sum().reset_index()
            live_agg.rename(columns={'fuel_used_l':'fuel_used_l_live'}, inplace=True)
            df = df.merge(live_agg, on='trip_id', how='left')
            df['fuel_used_l'] = df.get('fuel_used_l_live').combine_first(df['distance_km']*df['fuel_efficiency_l_per_km'])
            df.drop(columns=['fuel_used_l_live'], inplace=True, errors='ignore')
        else:
            df['fuel_used_l'] = df['distance_km'] * df['fuel_efficiency_l_per_km']

        self.fuel_data = df
        self._generate_alerts()

    # ---------------- Alert system ----------------
    def _generate_alerts(self):
        df = self.fuel_data.copy()
        df['alert_threshold'] = df['vehicle_type'].map(self.vehicle_thresholds).fillna(1.1)
        alerts = df[df['predicted_fuel_l'] > df['fuel_used_l'] * df['alert_threshold']]

        for _, row in alerts.iterrows():
            notification = {
                'notification_id': f"N{datetime.now().strftime('%f')}",
                'recipient_type': 'driver',
                'recipient_id': row.get('driver_id','unknown'),
                'channel': 'push',
                'subject': 'Fuel Alert',
                'message': f"Trip {row['trip_id']} has higher than expected fuel consumption.",
                'status': 'queued',
                'created_at': datetime.now()
            }
            self.notifications = pd.concat([self.notifications, pd.DataFrame([notification])], ignore_index=True)

        self.alerts = alerts

    # ---------------- Public methods ----------------
    def calculate_fuel(self):
        self._compute_fuel()
        return self.fuel_data

    def get_alerts(self):
        return self.alerts

    def update_segment(self, trip_id, fuel_used_l):
        segment = {'trip_id': trip_id, 'fuel_used_l': fuel_used_l}
        self.live_segments = pd.concat([self.live_segments, pd.DataFrame([segment])], ignore_index=True)
        print(f"Live segment updated for trip {trip_id}")
        return self.calculate_fuel()


# ---------------- Demo ----------------
if __name__ == "__main__":
    trips = pd.DataFrame([
        {'trip_id':'T1','vehicle_id':'V1','distance_km':100,'driver_id':'D1'},
        {'trip_id':'T2','vehicle_id':'V2','distance_km':150,'driver_id':'D2'}
    ])
    vehicles = pd.DataFrame([
        {'vehicle_id':'V1','baseline_kmpl':12,'vehicle_type':'van'},
        {'vehicle_id':'V2','baseline_kmpl':10,'vehicle_type':'truck'}
    ])
    fuel_preds = pd.DataFrame([
        {'trip_id':'T1','predicted_fuel_l':9},
        {'trip_id':'T2','predicted_fuel_l':16}
    ])
    thresholds = {'van': 1.05, 'truck': 1.15}

    agent = FuelAgent(trips, vehicles, fuel_preds, vehicle_thresholds=thresholds)

    print("Fuel Data:")
    print(agent.calculate_fuel()[['trip_id','distance_km','fuel_used_l','predicted_fuel_l']])

    print("\nFuel Alerts:")
    print(agent.get_alerts())


Fuel Data:
  trip_id  distance_km  fuel_used_l  predicted_fuel_l
0      T1          100     8.333333                 9
1      T2          150    15.000000                16

Fuel Alerts:
  trip_id vehicle_id  distance_km driver_id  fuel_efficiency_l_per_km  \
0      T1         V1          100        D1                  0.083333   

  vehicle_type  predicted_fuel_l  fuel_used_l  alert_threshold  
0          van                 9     8.333333             1.05  


C:\Users\ASUS\AppData\Local\Temp\ipykernel_23856\2509082229.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.notifications = pd.concat([self.notifications, pd.DataFrame([notification])], ignore_index=True)
